Парсинг отзывов, всего спаршено более 1000 отзывов. Результат - csv файл, где первая колонка - количество звезд, вторая - текст отзыва

In [ ]:
import requests, csv
from bs4 import BeautifulSoup
import re

URL = "https://beloris.ru/comment/type/kupit-maska-dlya-litsa"
HEADERS = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36', 'accept': '*/*'}
HOST = "https://beloris.ru/"
FILE = "revs.csv"

def get_html(url, params = None):
    r = requests.get(url, headers = HEADERS, params = params)
    return r

def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all("div", class_ = "opinion-product")
    reviews =[]
    for item in items:
        for i in range(1,6):
            class_rev = 'product-review is-rating-' + str(i)
            #print(class_rev)
            score = item.find('div', class_= class_rev)
            if score != None:
              break
        review = item.find('div', class_='opinion-product__text').find('div').get_text(strip=True)
        reviews.append({
            'score': i,
            'review': review
        })

    return reviews

def save_file(items, path):
    with open (path, 'w', newline = '') as f:
        writer = csv.writer(f, delimiter = ';')
        writer.writerow(['Оценка','Отзыв'])
        for item in items:
            writer.writerow([item['score'],item['review']])


def parse():
    html = get_html(URL)
    if html.status_code == 200:
        reviews = []
        reviews.extend(get_content(html.text))
        print(reviews)
        for page in range(1, 90):
            print('Парсинг страницы', page)
            html = get_html(URL, params={'page': page})
            p=get_content(html.text)
            reviews.extend(p)
        print(reviews)
        save_file(reviews, FILE)
    else:
        print('Error')


parse()

Сортировка отзывов по рейтингу

In [125]:
import operator, csv
import random
def get_sorted():
  with open('revs.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
      sortedlist = sorted(reader, key=operator.itemgetter(0), reverse=True)
  return sortedlist

Доп библиотеки

In [126]:
! pip install pymorphy2
! pip install sklearn
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Токенизация слов, приведение их к нижнему регистру и постановка в начальную форму

In [127]:
def norm_token_review(review):
  result = []
  words = [w.lower() for w in word_tokenize(review) if w.isalpha()]
  for word in words:
      result.append(morph.parse(word)[0].normal_form) 
  return result

def tokenize_normalize(list_revs):
  trueforms = []
  for item in list_revs:
    words = norm_token_review(item[1])
    trueforms.append([item[0], words])
  return trueforms

Из отсортированного и преобразованного списка берутся первые 40 элементов (положительных отзывов) и последние 40 элементов (отрицательных отзывов). На выходе - кортеж из двух списков

In [128]:
def almost_true():
  truelist=tokenize_normalize(get_sorted())
  tryy = []
  almost_good = truelist[:40]
  reversee = truelist[::-1]
  alm_bad = reversee[:40]
  return almost_good, alm_bad

Составление частотного словаря, удаление из него элементов, у которых частота употребления меньше 4, преобразование словаря в список. В итоге получаются два списка - один для слов из положительных отзывов, другой - наоборот

In [129]:
import collections
def get_freqs(list_name):
  sw = stopwords.words('russian')
  new_list = []
  freq = {}
  for review in list_name:
    new_list.extend(review[1])
  filter = [word for word in new_list if word not in sw]
  #print(filter)
  for word in filter:
    if word in freq:
        freq[word] += 1
    else:  
        freq[word] = 1
  return freq

def del_somefreqs(freq, min_freq = 4):
  new_freq = {}
  #print(freq)
  for key, value in freq.items():
    if value >= min_freq:
      new_freq[key] = value
  return new_freq 

def freq_words(freq):
  freq_list=[]
  for k in sorted(freq, key = freq.get, reverse = True):
    freq_list.append(k)
  return freq_list
  
v1, v2 = almost_true()
#print(good_words)
good_freqs = get_freqs(v1)
good_words = freq_words(del_somefreqs(good_freqs, 3))
bad_freqs = get_freqs(v2)
bad_words = freq_words(del_somefreqs(bad_freqs))
good_words, bad_words

(['маска',
  'кожа',
  'очень',
  'лицо',
  'хороший',
  'отличный',
  'увлажнять',
  'патч',
  'пахнуть',
  'понравиться',
  'тканевый',
  'поэтому',
  'приятный',
  'приятно',
  'это',
  'смывать',
  'отлично',
  'любить',
  'разглаживать',
  'рекомендовать',
  'запах',
  'всё',
  'эффект',
  'мягкий',
  'успокаивать',
  'сухой',
  'глаз',
  'сторона',
  'сыворотка',
  'вода',
  'серия',
  'неделя',
  'дарить',
  'лекало',
  'масочок',
  'увлажнить',
  'сползать',
  'напитать',
  'упаковка',
  'нежный',
  'процедура',
  'остаток',
  'липкий',
  'брать',
  'тонизировать',
  'заявить',
  'эластичность',
  'советовать',
  'стоить',
  'гладкий',
  'пропитать',
  'шея'],
 ['маска',
  'эффект',
  'кожа',
  'лицо',
  'это',
  'очень',
  'большой',
  'никакой',
  'запах',
  'всё',
  'вообще',
  'ужасный',
  'использование',
  'плюс',
  'липкий',
  'мочь',
  'нос',
  'купить',
  'минута',
  'увлажнение',
  'пора',
  'глаз',
  'любить',
  'заметить',
  'хороший',
  'стать',
  'применение',
  '

Преобразование списков во множества и нахождение тех элементов, которые есть только в положительном множестве и, наоборот, только в отрицательном

In [130]:
good_set = set(good_words) - set(bad_words)
bad_set = set(bad_words) - set(good_words)
good_set, bad_set

({'брать',
  'гладкий',
  'дарить',
  'заявить',
  'масочок',
  'мягкий',
  'напитать',
  'неделя',
  'нежный',
  'остаток',
  'отлично',
  'отличный',
  'патч',
  'пахнуть',
  'поэтому',
  'приятно',
  'приятный',
  'пропитать',
  'процедура',
  'разглаживать',
  'рекомендовать',
  'серия',
  'смывать',
  'советовать',
  'сползать',
  'стоить',
  'сторона',
  'сухой',
  'сыворотка',
  'тканевый',
  'тонизировать',
  'увлажнить',
  'увлажнять',
  'упаковка',
  'успокаивать',
  'шея',
  'эластичность'},
 {'аллергия',
  'аромат',
  'большой',
  'вообще',
  'дать',
  'держать',
  'жжение',
  'жидкость',
  'заметить',
  'использование',
  'красный',
  'купить',
  'минута',
  'мочь',
  'неприятный',
  'никакой',
  'нос',
  'нужно',
  'обычный',
  'ощущение',
  'первый',
  'плюс',
  'пора',
  'применение',
  'продукт',
  'просто',
  'разочарование',
  'самый',
  'сказать',
  'стать',
  'увлажнение',
  'ужас',
  'ужасный'})

Проверка 100 рандомных отзывов на то, положительный он или отрицательный. True - отзыв положительный (оценка 5 или 4), False - отрицательный (все остальные оценки). А также подсчет качества модели

In [136]:
def is_good_review(review, good_set, bad_set, good_freqs, bad_freqs):
  norm_rev = norm_token_review(review)
  norm_set = set(norm_rev)
  is_good = norm_set & good_set
  is_bad = norm_set & bad_set
  #print(review)
  '''
  freq_rev = get_freqs([[0,norm_rev]])
  print(review)
  print(freq_rev)
  print("++++++++++++++")
  for word in is_good:
    if word in good_freqs:
      print(word, good_freqs[word])
  print("--------------")
  for word in is_bad:
    if word in bad_freqs:
      print(word, bad_freqs[word])
  '''
  if len(is_good) >= len(is_bad):
    return True
  else:
    return False

reviews = get_sorted()
y_pred = []
y_true = []
for i in range (100):
  rnd = random.choice(reviews)
  review = rnd[1]
  score = rnd[0]
  y_true.append(int(score) > 3)
  y_pred.append(is_good_review(review, good_set, bad_set, good_freqs, bad_freqs))
print(y_true)
print(y_pred)
accuracy_score(y_true, y_pred)


[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, False, True, False, False, True, True, True, True, True, True, False, True, True, True, True, False, True, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, False, False, False, True, Tru

0.8

Более информативно по 10 рандомным отзывам. Предположительная оценка: если True, то скорее положительная, False - скорее отрицательная

In [123]:
reviews = get_sorted()
y_pred = []
y_true = []
for i in range (10):
  rnd = random.choice(reviews)
  review = rnd[1]
  score = rnd[0]
  y_true.append(score)
  y_pred.append(is_good_review(review, good_set, bad_set, good_freqs, bad_freqs))
for i in range(len(y_true)):
  print("Настоящая оценка - ", y_true[i], ', Предположительно - ', y_pred[i])

Настоящая оценка -  5 , Предположительно -  True
Настоящая оценка -  5 , Предположительно -  False
Настоящая оценка -  5 , Предположительно -  True
Настоящая оценка -  5 , Предположительно -  False
Настоящая оценка -  5 , Предположительно -  True
Настоящая оценка -  5 , Предположительно -  True
Настоящая оценка -  5 , Предположительно -  True
Настоящая оценка -  5 , Предположительно -  True
Настоящая оценка -  5 , Предположительно -  True
Настоящая оценка -  4 , Предположительно -  False


Способы улучшения:
1. Для лучшего результата отсортировать слова при помощи расстояния Ливенштейна, далее использовать стемминг для избавления от однокоренных слов разных частей речи (напр. отлично и отличный). Тогда accuracy станет более точной.
2. Отсортировать отзывы не просто по первому столбцу(рейтинг отзыва), но еще и по длине отзыва. Чем больше слов, тем, скорее всего, точнее будут наши множества.